# ***IMDb, analizando la base de datos***

Importamos las librerías necesarias;

In [3]:
!pip install prettytable

In [4]:
import os
import tarfile
import hashlib
import urllib.request
import pandas as pd
from datetime import datetime
from collections import defaultdict
from pathlib import Path
import stat
import requests
from bs4 import BeautifulSoup
from prettytable import PrettyTable

1. Función para descargar el Large Movie Review Database a un directorio dado, si el archivo ya existe, omitir la descarga.

In [5]:
def download_database(url, directory):
    archivo = os.path.join(directorio, os.path.basename(url))
    os.makedirs(os.path.dirname(archivo), exist_ok=True)

    if not os.path.exists(archivo): # Si el archivo no existe, descargarlo
        urllib.request.urlretrieve(url, archivo)
        print("Descarga completada.")
    else: # Si el archivo ya existe, imprimir un mensaje
        print("El archivo ya existe. Se omite la descarga.")
    return archivo

2. Función que descomprime el archivo en el mismo directorio, de tal manera que obtengas el directorio de la base de datos y el archivo con extensión `.tar.gz` en el mismo directorio.

In [6]:
def decompress_file(file, directory):
    try:
        with tarfile.open(file, "r:gz") as tar:
            tar.extractall(path=directory)
        print(f"Descomprimido en {directory}.")
    except EOFError:
        print("Por favor, vuelve a descargar el archivo e inténtalo de nuevo.")
        return

3. A continuación se implementan las funciones necesarias para encontrar las 10 películas peor calificadas en promedio y las 10 películas mejor calificadas en promedio del directorio `train`.

In [ ]:
def leer_puntuaciones(directorio):
    """Lee los archivos en el directorio especificado y devuelve un diccionario con las puntuaciones por identificador de película."""
    puntuaciones = defaultdict(list)

    # Recorrer archivos en subdirectorios de train (ej: train/pos y train/neg)
    for subdir, _, files in os.walk(directorio):
        for file_name in files:
            file_path = Path(subdir) / file_name
            try:
                # Extraer identificador y puntuación (asumiendo formato 'id_score.txt')
                identificador, puntuacion_str = file_name.split('_')
                puntuacion = int(puntuacion_str.split('.')[0])
                puntuaciones[identificador].append(puntuacion)
            except (ValueError, IndexError):
                # Omitir archivos que no cumplan con el formato esperado
                print(f"Archivo omitido: {file_name} (no tiene un formato válido)")
                continue

    return puntuaciones

def calcular_promedios(puntuaciones):
    """Calcula el promedio de puntuaciones por identificador de película."""
    promedios = {identificador: sum(scores) / len(scores) for identificador, scores in puntuaciones.items()}
    return promedios

def obtener_top_n(promedios, n=10, mejor=True):
    """Obtiene las n mejores o peores películas en función del promedio de puntuaciones."""
    return sorted(promedios.items(), key=lambda x: x[1], reverse=mejor)[:n]

def procesar_url(url):
    """Extrae la parte de la URL que contiene el título de la película."""
    return url.split('/usercomments')[0]

def obtener_nombre_pelicula(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 403:
        return "Acceso denegado (403 Forbidden)"

    soup = BeautifulSoup(response.content, 'html.parser')
    title_tag = soup.find('title')
    if title_tag:
        return title_tag.text.replace(' - IMDb', '').strip()
    else:
        return "Título no encontrado"

def crear_dataframe(peliculas, urls_path):
    """Crea un DataFrame con la información de las películas."""
    urls = Path(urls_path).read_text().splitlines()
    data = []

    for identificador, promedio in peliculas:
        url = procesar_url(urls[int(identificador) - 1])
        data.append({'url': url, 'Película': obtener_nombre_pelicula(url), 'Promedio': promedio})

    return pd.DataFrame(data)

def imprimir_peliculas(titulo, peliculas):
        '''Imprime en consola el título y la lista de películas con su promedio y URL en formato de tabla.'''
        table = PrettyTable()
        table.field_names = ["Película", "Promedio", "URL"]
        for pelicula in peliculas:
            table.add_row([pelicula['Película'], f"{pelicula['Promedio']:.2f}", pelicula['url']])
        print(titulo)
        print(table)

def peliculas(directorio_pos, directorio_neg, urls_pos, urls_neg):
        # Leer puntuaciones
        puntuaciones_pos = leer_puntuaciones(directorio_pos)
        puntuaciones_neg = leer_puntuaciones(directorio_neg)

        # Calcular promedios
        promedios_p = calcular_promedios(puntuaciones_pos)
        promedios_n = calcular_promedios(puntuaciones_neg)

        # Obtener las 10 mejores y peores películas
        mejores_peliculas = obtener_top_n(promedios_p)
        peores_peliculas = obtener_top_n(promedios_n, mejor=False)

        # Crear DataFrames
        df_mejores = crear_dataframe(mejores_peliculas, urls_pos)
        df_peores = crear_dataframe(peores_peliculas, urls_neg)

        while len(df_mejores) < 10:
            n_a_row = pd.DataFrame({'url': [''], 'Película': ['N/A'], 'Promedio': [0]})
            df_mejores = pd.concat([df_mejores, n_a_row], ignore_index=True)

        while len(df_peores) < 10:
            n_a_row = pd.DataFrame({'url': [''], 'Película': ['N/A'], 'Promedio': [0]})
            df_peores = pd.concat([df_peores, n_a_row], ignore_index=True)

        imprimir_peliculas("Las 10 películas mejor calificadas:\n", df_mejores.to_dict(orient='records'))
        imprimir_peliculas("Las 10 películas peor calificadas:\n", df_peores.to_dict(orient='records'))

In [ ]:
# Directorio principal de reseñas y archivo de URLs
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
directorio = "./downloads/basedatos/aclImdb"
directorio_pos = './downloads/basedatos/aclImdb/aclImdb/train/pos'
directorio_neg = './downloads/basedatos/aclImdb/aclImdb/train/neg'
urls_pos = './downloads/basedatos/aclImdb/aclImdb/train/urls_pos.txt'
urls_neg = './downloads/basedatos/aclImdb/aclImdb/train/urls_neg.txt'
archivo = download_database(url, directorio)
decompress_file(archivo, directorio)
peliculas(directorio_pos, directorio_neg, urls_pos, urls_neg)